In [23]:
import smtplib
from email.mime.text import MIMEText
from email.header import Header
import pandas as pd
import time
%run schedule_reader.ipynb
raw_courses=read_schedule()
print(pd.Timestamp(raw_courses[0]['class_time_str']))
print(pd.Timestamp.now())

2025-10-16 09:50:00
2025-10-16 15:58:51.997994


In [24]:
# --- 邮件配置 ---
# SMTP 服务器配置
smtp_server = 'smtp.qq.com'  # 例如: smtp.qq.com
smtp_port = 465              # SSL 端口
# 发件人账户信息
sender_email = '2200355981@qq.com'      # 你的邮箱地址
sender_password = 'abcdefghijklmn'     # 你的邮箱授权码，而不是登录密码！

In [25]:
def send_reminder(receiver_email, course_name, class_time_str, location):
    message = MIMEText(
        f'【课程名称】{course_name}'
        f'【课程时间】{class_time_str}'
        f'【课程地点】{location}'
    )
    message['From'] = Header(sender_email)
    message['To'] = Header(receiver_email)
    message['Subject'] = Header(f'【课程提醒】{course_name}{class_time_str}{location}')

    try:
        server = smtplib.SMTP('smtp.qq.com', 465)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        print('邮件发送成功')
    except smtplib.SMTPException as e:
        print(f'邮件发送失败: {e}')
    finally:
        server.quit()

In [26]:
receiver_email = '10244801409@stu.ecnu.edu.cn'      # 在这填写接收reminder的邮箱
check_interval = 60     #设定检查时间
def time_checker(check_interval, raw_courses):
    while True:
        for x in raw_courses:
            if pd.Timestamp.now() >= pd.Timestamp(x['class_time_str']) - pd.Timedelta(minutes=10) and not x['is_reminded']:
                send_reminder(receiver_email, x['course_name'], x['class_time_str'], x['location'])
        time.sleep(check_interval)